In [5]:
# -*- coding: utf-8 -*-
import os, re, time
from glob import glob
from pathlib import Path
import pandas as pd
import win32com.client as win32

DATA_DIR = Path("무/무")        # .xls 들이 있는 폴더
OUT_DIR  = Path("무_csv")    # 결과 csv 저장 폴더
TMP_DIR  = OUT_DIR / "_tmp_xlsx"  # 임시 .xlsx 저장 폴더 (OneDrive 회피)
OUT_DIR.mkdir(parents=True, exist_ok=True)
TMP_DIR.mkdir(parents=True, exist_ok=True)

def clean_df(df: pd.DataFrame) -> pd.DataFrame:
    df.columns = df.columns.str.strip()
    cols = df.columns.tolist()
    print("DEBUG: 현재 파일 컬럼 =", cols)   # <-- 문제 추적용 출력

    # 날짜 컬럼 자동 찾기
    if "구분" in cols:
        date_col = "구분"
    elif "일자" in cols:
        date_col = "일자"
    else:
        raise KeyError("날짜 컬럼('구분' 또는 '일자')을 찾을 수 없습니다")

    # 숫자 컬럼 자동 찾기
    num_candidates = ["평균", "전통시장", "유통업체", "평균가"]
    num_cols = [c for c in num_candidates if c in cols]

    # ---- (추가) 단위 필터링 ----
    if "단위" in cols:
        before_len = len(df)
        df = df[df["단위"].astype(str).str.contains("20", na=False)].copy()
        print(f"단위 필터링: {before_len}행 -> {len(df)}행 (10키로만 남김)")

    # 날짜 변환
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
    df = df[df[date_col].notna()].copy()

    # 숫자 변환
    for c in num_cols:
        df[c] = (
            df[c].astype(str)
                 .str.replace(",", "", regex=False)
                 .str.strip()
        )
        df[c] = pd.to_numeric(df[c], errors="coerce")

    return df[[date_col] + num_cols]

def convert_all():
    xls_files = sorted(glob(str(DATA_DIR / "*.xls")))
    if not xls_files:
        raise FileNotFoundError(f"{DATA_DIR.resolve()} 안에 .xls 파일이 없습니다.")

    # Excel 인스턴스 1개만 사용
    excel = win32.gencache.EnsureDispatch("Excel.Application")
    excel.Visible = False
    excel.DisplayAlerts = False

    try:
        for f in xls_files:
            src = Path(f)
            tmp_xlsx = TMP_DIR / (src.stem + ".__tmp__.xlsx")  # 확실히 고유한 임시 파일명
            out_csv  = OUT_DIR / (src.stem + ".csv")
            try:
                # 기존 임시파일 있으면 삭제
                if tmp_xlsx.exists():
                    try: tmp_xlsx.unlink()
                    except: pass

                # 읽기 전용으로 열기 (잠금 회피)
                wb = excel.Workbooks.Open(str(src.resolve()), ReadOnly=True)
                # 완전한 경로로 xlsx 저장 (OneDrive 기본 경로로 새는 것 방지)
                wb.SaveAs(str(tmp_xlsx.resolve()), FileFormat=51)  # 51 = xlsx
                wb.Close(SaveChanges=False)

                # pandas로 정리
                df = pd.read_excel(tmp_xlsx, header=1, engine="openpyxl")
                df = clean_df(df)
                df.to_csv(out_csv, index=False, encoding="utf-8-sig")
                print(f"[OK] {src.name} -> {out_csv.name}")

            except Exception as e:
                print(f"[ERROR] 변환 실패: {src} -> {e}")
            finally:
                # 임시 xlsx 삭제 시도
                if tmp_xlsx.exists():
                    try: tmp_xlsx.unlink()
                    except: pass
    finally:
        # Excel 종료
        try:
            excel.Quit()
        except:  # 이미 종료돼도 무시
            pass

if __name__ == "__main__":
    convert_all()


DEBUG: 현재 파일 컬럼 = ['일자', '시장', '품목', '단위', '등급', '평균가']
단위 필터링: 104행 -> 104행 (10키로만 남김)
[OK] 가격정보 (1).xls -> 가격정보 (1).csv
DEBUG: 현재 파일 컬럼 = ['일자', '시장', '품목', '단위', '등급', '평균가']
단위 필터링: 101행 -> 101행 (10키로만 남김)
[OK] 가격정보 (10).xls -> 가격정보 (10).csv
DEBUG: 현재 파일 컬럼 = ['일자', '시장', '품목', '단위', '등급', '평균가']
단위 필터링: 97행 -> 97행 (10키로만 남김)
[OK] 가격정보 (11).xls -> 가격정보 (11).csv
DEBUG: 현재 파일 컬럼 = ['일자', '시장', '품목', '단위', '등급', '평균가']
단위 필터링: 104행 -> 104행 (10키로만 남김)
[OK] 가격정보 (12).xls -> 가격정보 (12).csv
DEBUG: 현재 파일 컬럼 = ['일자', '시장', '품목', '단위', '등급', '평균가']
단위 필터링: 101행 -> 101행 (10키로만 남김)
[OK] 가격정보 (2).xls -> 가격정보 (2).csv
DEBUG: 현재 파일 컬럼 = ['일자', '시장', '품목', '단위', '등급', '평균가']
단위 필터링: 105행 -> 105행 (10키로만 남김)
[OK] 가격정보 (3).xls -> 가격정보 (3).csv
DEBUG: 현재 파일 컬럼 = ['일자', '시장', '품목', '단위', '등급', '평균가']
단위 필터링: 103행 -> 103행 (10키로만 남김)
[OK] 가격정보 (4).xls -> 가격정보 (4).csv
DEBUG: 현재 파일 컬럼 = ['일자', '시장', '품목', '단위', '등급', '평균가']
단위 필터링: 99행 -> 99행 (10키로만 남김)
[OK] 가격정보 (5).xls -> 가격정보 (5).csv
DEBUG: 현재 파일 컬럼 = ['일자